# Hubbard model
In this example, we look into another kind of insulator, the Mott-Hubbard insulator, again in the 1d mono-atomic chain. In which the  electrons are localized due to the electron-electron Coulomb interaction. We consider the Hubbard model:
$$H_{elec}=\sum_{ij}t_{ij} c^{\dagger}_i c_j + c.c. + \frac{1}{2}\sum_i U n_{i\uparrow} n_{i\downarrow} $$.

In [1]:
%matplotlib inline
import numpy as np
from  minimulti.electron.Hamiltonian import etb_model, atoms_model
from minimulti.electron.basis2 import BasisSet, gen_basis_set, atoms_to_basis
from ase.atoms import Atoms
import matplotlib.pyplot as plt


def run_1d_chain_Hubbard(t=1, U=9.9, spinat=[1,-1], nel=2, v=0.0):
    # generate structure.
    atoms=Atoms(symbols='H', positions=[(0,0,0)], cell=[1,1,1])
    # generate basis set
    bset= atoms_to_basis(atoms, basis_dict={'H': ('s',)}, nspin=2)
    # generate model
    mymodel=atoms_model(atoms=atoms, basis_set=bset, nspin=2)

    # onsite energy (tune v to make electron localized on one site)
    mymodel.set_onsite(v,0)
    mymodel.set_onsite(v,1)

    # 1D hopping along z. Format: t, i, j, R.
    # spin up
    mymodel.set_hop(t, 0,0, [0,0,1])
    # spin down
    mymodel.set_hop(t, 1,1, [0,0,1])

    mymodel.set(nel=nel, mixing=0.5, tol_energy=1e-8, tol_rho=1e-8, sigma=0.05)
    mymodel.set_initial_spin(spinat)
    mymodel.set_Hubbard_U(Utype='Kanamori', Hubbard_dict={'H':{'L':0, 'U':U, 'J':0}})

    # K mesh.
    mymodel.set_kmesh([1,1,30])

    # Scf solve
    mymodel.scf_solve(print_iter_info=False)
    efermi=mymodel.get_fermi_level()

    kpt_x=np.arange(0, 1.01,0.01)
    kpts=[np.array([0,0,1]) * x for x in kpt_x]
    evalues, evecs=mymodel.solve_all(k_list=kpts)

    for i in range(mymodel._norb):
        plt.plot(kpt_x, evalues[i,:])
    plt.xlabel('k-point')
    plt.ylabel('Energy')
    plt.axhline(efermi, linestyle='--', color='gray')

In this system, each site has one orbital and thus can have two electrons. The electrons should be able to hop to the neighboring site. However, if there is Coulomb interaction between electrons on the same site, the electrons would need extra energy to be at the same site. If this energy is larger than the kinetic energy of the electron, the electrons will be localized. We can tune the U value below to see the change between metal and insulating state.

In [2]:
#run_1d_chain_Hubbard(t=1, U=5.300, spinat=[1.0], nel=1.0, v=0.0)
from ipywidgets import FloatSlider, interact, fixed
interact(run_1d_chain_Hubbard, t=fixed(1), U=(0.0, 8.0, 1.0), spinat=fixed([1.0]), nel=fixed(1.0), v=fixed(0.0));

interactive(children=(FloatSlider(value=8.0, description='U', max=8.0, step=1.0), Output()), _dom_classes=('wi…